In [1]:
import pandas as pd
import geopandas as gpd
import folium, os

In [59]:
def set_color_tuple(row):
    total = sum(row.iloc[:][1:4])
    color_tup = (round(255*row.iloc[:][1]/total, 1), 
                 round(255*row.iloc[:][3]/total, 1),
                 round(255*row.iloc[:][2]/total, 1))
    return color_tup

def rgb_to_hex(rgb_tup):
    rgb_tup = tuple(round(x) for x in rgb_tup)
    return '#%02x%02x%02x' % rgb_tup


def get_data_by_state_year(state='MN', year=2012):
    filename = f'{state}{year}.txt'
    
    dat = pd.read_csv(f'../data/{filename}').drop('Unnamed: 4', axis=1)
    dat['rgb_color'] = dat.apply(lambda row: set_color_tuple(row), axis=1)
    dat['hex_color'] = dat.apply(lambda row: rgb_to_hex(row.rgb_color), axis=1)
    dat['Year'] = year
    dat['State'] = state
    dat = dat.rename(columns={f'{year} US Presidential Election':'County',
                             dat.columns[1]:'Republican',
                             dat.columns[2]:'Democrat',
                             dat.columns[3]:'Other'})
    
    cols = dat.columns.tolist()
    cols = cols[-2:] + cols[:-2]
    
    return dat[cols]

def identify_results_files():
    
    listdir = os.listdir('../data')
    
    filenames = [filename for filename in listdir if len(filename)==10]
    
    return filenames

In [60]:
get_data_by_state_year()

,Year,State,County,Republican,Democrat,Other,rgb_color,hex_color
0,2012,MN,Aitkin,4533,4412,197,"(126.4, 5.5, 123.1)",#7e067b
1,2012,MN,Anoka,93430,88614,4421,"(127.8, 6.0, 121.2)",#800679
2,2012,MN,Becker,9204,6829,349,"(143.3, 5.4, 106.3)",#8f056a
3,2012,MN,Beltrami,9637,11818,596,"(111.4, 6.9, 136.7)",#6f0789
4,2012,MN,Benton,10849,8173,597,"(141.0, 7.8, 106.2)",#8d086a
...,...,...,...,...,...,...,...,...
82,2012,MN,Watonwan,2517,2494,133,"(124.8, 6.6, 123.6)",#7d077c
83,2012,MN,Wilkin,1884,1258,80,"(149.1, 6.3, 99.6)",#950664
84,2012,MN,Winona,11480,14980,772,"(107.5, 7.2, 140.3)",#6c078c
85,2012,MN,Wright,40466,25741,1609,"(152.2, 6.1, 96.8)",#980661


In [103]:
# Create initial dictionary so that I can create initial DataFrame
# so that I can concatenate to that DataFrame
setup_dict = dict.fromkeys(get_data_by_state_year().columns)
setup_dict['Year'] = []

results_df = pd.DataFrame.from_dict(setup_dict)

results_df['Year'] = pd.to_numeric(results_df['Year'], downcast='integer')

# results_df

results_filenames = identify_results_files()

counter = 1
print('File number: ', counter)

for filename in results_filenames:
    if counter % 25 == 0:
        print('File number: ', counter)
    counter += 1
    
    state_abbr = filename[:2]
    year = filename[2:6]
#     print(state_abbr, year)
    
    df = get_data_by_state_year(state_abbr, year)
#     print(df.head(1))
    
    results_df = results_df.append(df, ignore_index=True)
    results_df.reset_index(drop=True,inplace=True)
#     print(results_df.tail(5))
    

print('File number: ', counter-1)
results_df = results_df.sort_values(by=['Year', 'State', 'County'])
results_df.reset_index(drop=True,inplace=True)

File number:  1
File number:  25
File number:  50
File number:  75
File number:  100
File number:  125
File number:  150
File number:  175
File number:  200
File number:  225
File number:  250
File number:  275
File number:  300
File number:  325
File number:  350
File number:  375
File number:  400
File number:  425
File number:  450
File number:  475
File number:  500
File number:  525
File number:  550
File number:  575
File number:  600
File number:  625
File number:  650
File number:  675
File number:  700
File number:  713


In [92]:
len(results_filenames)

713

In [104]:
results_df

,Year,State,County,Republican,Democrat,Other,rgb_color,hex_color
0,1960,AK,District 1,180,257,0,"(105.0, 0.0, 150.0)",#690096
1,1960,AK,District 10,11119,9581,0,"(137.0, 0.0, 118.0)",#890076
2,1960,AK,District 11,568,634,0,"(120.5, 0.0, 134.5)",#780086
3,1960,AK,District 12,1005,1120,0,"(120.6, 0.0, 134.4)",#790086
4,1960,AK,District 13,631,867,0,"(107.4, 0.0, 147.6)",#6b0094
...,...,...,...,...,...,...,...,...
43956,2012,WY,Sweetwater,11428,4774,693,"(172.5, 10.5, 72.1)",#ac0a48
43957,2012,WY,Teton,4858,6213,393,"(108.1, 8.7, 138.2)",#6c098a
43958,2012,WY,Uinta,6615,1628,296,"(197.5, 8.8, 48.6)",#c60931
43959,2012,WY,Washakie,3014,794,136,"(194.9, 8.8, 51.3)",#c30933


In [ ]:
dat = pd.read_csv('../data/USA2012.txt').drop('Unnamed: 4', axis=1)

dat['rgb_color'] = dat.apply(lambda row: set_color_tuple(row), axis=1)
dat['hex_color'] = dat.apply(lambda row: rgb_to_hex(row.rgb_color), axis=1)

dat.head()

In [ ]:
import json
import requests


url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
us_states = f'{url}/us-states.json'

geo_json_data = json.loads(requests.get(us_states).text)

In [ ]:
m = folium.Map([43, -100], tiles='cartodbpositron', zoom_start=4)

folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        'fillColor': dat['hex_color'][0],
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(m)

m

In [ ]:
geo_json_data['features'][0]['properties']['name']

In [ ]:
dat.head()

In [ ]:
geo_json_data['features'][0]['fillColor'] = 'black'

In [ ]:
geo_json_data.keys()

In [ ]:
for state_num in range(len(geo_json_data['features'])):
    state_name = geo_json_data['features'][state_num]['properties']['name']
    geo_json_data['features'][state_num]['fillColor'] = (dat.loc[dat['2012 US Presidential Election']==state_name].reset_index(drop=True)).iloc[0].hex_color

In [ ]:
def filter_state(state_name='Minnesota'):
    for num in range(len(geo_json_data['features'])):
        if state_name == geo_json_data['features'][num]['properties']['name']:
            return geo_json_data['features'][num]

In [ ]:
filter_state('Minnesota')

In [ ]:
district_json_data = {'type': 'FeatureCollection'}
filtered_value = filter_state('Minnesota')
district_json_data.update({'features':[filtered_value]})

district_json_data

In [ ]:
m = folium.Map([46, -93], tiles='cartodbpositron', zoom_start=7)

def add_district_to_map(district_data):
    folium.GeoJson(
        district_data,
        style_function=lambda feature: {
            'fillColor' : district_data['features'][0]['fillColor'],
            'weight': 0
        }
    ).add_to(m)

In [ ]:
add_district_to_map(district_json_data)

m

In [ ]:
url = 'https://github.com/python-visualization/folium/raw/master/tests'
us_counties = f'{url}/us-counties.json'

county_geo_json_data = json.loads(requests.get(us_counties).text)

In [ ]:
def filter_state_counties(state_name='Minnesota'):
    for num in range(len(county_geo_json_data['features'])):
        if state_name == county_geo_json_data['features'][num]['properties']['name']:
            return county_geo_json_data['features'][num]

In [ ]:
county_geo_json_data

In [ ]:
m = folium.Map([43, -100], tiles='cartodbpositron', zoom_start=5)

folium.GeoJson(
    county_geo_json_data,
    style_function=lambda feature: {
        'fillColor': 'green',
        'color': 'grey',
        'weight': 1,
        'dashArray': '1, 5'
    }
).add_to(m)

m

In [ ]:
len(county_geo_json_data['features'])